In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

1. Data Ingestion

In [2]:
# Step 1: Load the dataset
df = pd.read_csv("/content/Uncleaned_DS_jobs.csv")
print("Initial shape:", df.shape)
print(df.info())

Initial shape: (672, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              672 non-null    int64  
 1   Job Title          672 non-null    object 
 2   Salary Estimate    672 non-null    object 
 3   Job Description    672 non-null    object 
 4   Rating             672 non-null    float64
 5   Company Name       672 non-null    object 
 6   Location           672 non-null    object 
 7   Headquarters       672 non-null    object 
 8   Size               672 non-null    object 
 9   Founded            672 non-null    int64  
 10  Type of ownership  672 non-null    object 
 11  Industry           672 non-null    object 
 12  Sector             672 non-null    object 
 13  Revenue            672 non-null    object 
 14  Competitors        672 non-null    object 
dtypes: float64(1), int64(2), object(12)
memory usage:

In [3]:
df.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [4]:
df.isnull().sum()

,0
index,0
Job Title,0
Salary Estimate,0
Job Description,0
Rating,0
Company Name,0
Location,0
Headquarters,0
Size,0
Founded,0


In [5]:
df.describe()

,index,Rating,Founded
count,672.000000,672.000000,672.000000
mean,335.500000,3.518601,1635.529762
std,194.133974,1.410329,756.746640
min,0.000000,-1.000000,-1.000000
25%,167.750000,3.300000,1917.750000
50%,335.500000,3.800000,1995.000000
75%,503.250000,4.300000,2009.000000
max,671.000000,5.000000,2019.000000


2. Deduplication

In [9]:
# step 2: Remove duplicates
df = df.drop_duplicates()

In [11]:
# Check for redundancy before dropping
redundant_loc = (df['Location'] == df['Headquarters']).sum()
print("Rows where Location = Headquarters:", redundant_loc)

# Now drop the redundant 'Headquarters' column
df = df.drop(columns=['Headquarters'])

KeyError: 'Headquarters'

In [12]:
df.shape

(672, 14)

In [14]:
# Rename columns
df.rename(columns={'Job Title': 'Job_Title', 'Salary Estimate': 'Salary_Estimate'}, inplace=True)

3. Column Management

In [15]:
# Drop the 'index' column if it exists
if 'index' in df.columns:
    df.drop(columns=['index'], inplace=True)


In [16]:
# Replace other -1 values with NaN
df['Founded'] = df['Founded'].replace(-1, np.nan)
df['Competitors'] = df['Competitors'].replace('-1', np.nan)
df['Size'] = df['Size'].replace('-1', np.nan)
df['Type of ownership'] = df['Type of ownership'].replace('-1', np.nan)
df['Industry'] = df['Industry'].replace('-1', np.nan)
df['Sector'] = df['Sector'].replace('-1', np.nan)
df['Revenue'] = df['Revenue'].replace('-1', np.nan)

# Ensure the 'Rating' column is numeric
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')

# Replace -1.0 (as float) with NaN
df['Rating'] = df['Rating'].replace(-1.0, np.nan)


In [ ]:
# Clean 'Salary Estimate'
df['Salary_Estimate'] = (
    df['Salary_Estimate']
    .str.replace(r'\(.*?\)', '', regex=True)
    .str.replace('$', '', regex=False)
    .str.replace('K', '', regex=False)
    .str.strip()
)
df[['min_salary', 'max_salary']] = df['Salary_Estimate'].str.split('-', expand=True)
df['min_salary'] = pd.to_numeric(df['min_salary'], errors='coerce')
df['max_salary'] = pd.to_numeric(df['max_salary'], errors='coerce')
df['avg_salary'] = df[['min_salary', 'max_salary']].mean(axis=1)

In [17]:
df.tail()

,Job_Title,Salary_Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
667,Data Scientist,$105K-$167K (Glassdoor est.),Summary\n\nWe’re looking for a data scientist ...,3.6,TRANZACT\n3.6,"Fort Lee, NJ",1001 to 5000 employees,1989.0,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,NaN
668,Data Scientist,$105K-$167K (Glassdoor est.),Job Description\nBecome a thought leader withi...,NaN,JKGT,"San Francisco, CA",NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,Data Scientist,$105K-$167K (Glassdoor est.),Join a thriving company that is changing the w...,NaN,AccessHope,"Irwindale, CA",NaN,NaN,NaN,NaN,NaN,NaN,NaN
670,Data Scientist,$105K-$167K (Glassdoor est.),100 Remote Opportunity As an AINLP Data Scient...,5.0,ChaTeck Incorporated\n5.0,"San Francisco, CA",1 to 50 employees,NaN,Company - Private,Advertising & Marketing,Business Services,$1 to $5 million (USD),NaN
671,Data Scientist,$105K-$167K (Glassdoor est.),Description\n\nThe Data Scientist will be part...,2.7,1-800-Flowers\n2.7,"New York, NY",1001 to 5000 employees,1976.0,Company - Public,Wholesale,Business Services,$1 to $2 billion (USD),NaN


In [18]:
df.shape

(672, 13)

4. Missing Value Handling

In [19]:
# Optional: check missing values
print(df[['Rating', 'Size', 'Founded','Type of ownership','Industry','Sector','Revenue','Competitors']].isna().sum())

Rating                50
Size                  27
Founded              118
Type of ownership     27
Industry              71
Sector                71
Revenue               27
Competitors          501
dtype: int64


In [20]:
#the missing values in competitors is more than 70% so drop Cometitors table
df = df.drop(columns=['Competitors'])

In [21]:
# Drop rows with NaN in specific columns
df = df.dropna(subset=['Rating', 'Founded', 'Size','Type of ownership','Industry','Sector','Revenue'])

In [22]:
df.shape

(550, 12)

## 5. Data Type Correction & 6. Format Standardization

In [25]:
text_columns = ['Job_Title', 'Location', 'Company Name', 'Size', 'Type of ownership',
                'Industry', 'Sector', 'Revenue']

for col in text_columns:
    df[col] = df[col].astype(str).str.strip().str.lower()

In [24]:
# Map categorical variants (example on 'Type of ownership')
df['Type of ownership'] = df['Type of ownership'].replace({
    'company - private': 'private',
    'private company': 'private',
    'company - public': 'public',
    'nonprofit organization': 'nonprofit'
})


In [ ]:
# Remove newline characters from 'Job Description' and 'Company Name'
df['Job Description'] = df['Job Description'].str.replace('\n', ' ', regex=False).str.strip()
df['Company Name'] = df['Company Name'].str.replace('\n', ' ', regex=False).str.strip()

In [26]:
df.tail()

,Job_Title,Salary_Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
663,data scientist,$105K-$167K (Glassdoor est.),Job Description\nEducation and Experience:\nAt...,4.1,a-line staffing solutions\n4.1,"durham, nc",501 to 1000 employees,2004.0,private,staffing & outsourcing,business services,unknown / non-applicable
665,data scientist,$105K-$167K (Glassdoor est.),Criterion Systems seeks a Data Scientist to su...,3.8,"criterion systems, inc.\n3.8","vienna, va",201 to 500 employees,2005.0,private,it services,information technology,$50 to $100 million (usd)
666,data scientist,$105K-$167K (Glassdoor est.),About Foundation Medicine:\n\nFoundation Medic...,4.0,foundation medicine\n4.0,"boston, ma",1001 to 5000 employees,2010.0,public,biotech & pharmaceuticals,biotech & pharmaceuticals,$100 to $500 million (usd)
667,data scientist,$105K-$167K (Glassdoor est.),Summary\n\nWe’re looking for a data scientist ...,3.6,tranzact\n3.6,"fort lee, nj",1001 to 5000 employees,1989.0,private,advertising & marketing,business services,unknown / non-applicable
671,data scientist,$105K-$167K (Glassdoor est.),Description\n\nThe Data Scientist will be part...,2.7,1-800-flowers\n2.7,"new york, ny",1001 to 5000 employees,1976.0,public,wholesale,business services,$1 to $2 billion (usd)


7. Save the cleaned_DS_Jobs

In [27]:
# Save the DataFrame to a CSV file before downloading
df.to_csv("cleaned_DS_jobs.csv", index=False) # index=False prevents writing the DataFrame index as a column

from google.colab import files
files.download("cleaned_DS_jobs.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>